In [6]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

from decimal import Decimal
from json import load as json_load
from operator import itemgetter
from os import environ
from os.path import join
from datetime import datetime, timedelta
from pprint import pprint
from time import sleep

from crocket.sql.sql import Database
from crocket.utilities.passcode import AESCipher
from crocket.utilities.credentials import get_credentials
from crocket.bittrex.bittrex2 import Bittrex

In [7]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = 'localhost'
DATABASE_NAME = 'PRODUCTION40'

In [8]:
BITTREX_CREDENTIALS_PATH = '/home/b3arjuden/bittrex_credentials.json'

with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    credentials = json_load(f)

In [9]:
b = Bittrex(api_key=credentials.get('key'), api_secret=credentials.get('secret'), api_version='v1.1')

In [10]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

In [11]:
mins = 2
time_str = datetime.strftime(datetime.now() - timedelta(minutes=mins), "%Y-%m-%d %H:%M:%S")

In [13]:
buy_volume = {}

skip_list = ['BTC-ETH', 'BTC-LTC', 'BTC-BCC', 'BTC-XVG', 'BTC-XLM', 'BTC-RDD']

while not buy_volume:

    db = Database(hostname=HOSTNAME,
                  username=USERNAME,
                  password=PASSCODE,
                  database_name=DATABASE_NAME)
    
    tables = db.get_all_tables()
    
    for table in tables:

        data = db.select_query(table, '*', 'where time > "{}"'.format('2017/12/21 12:30:00'))
        buy_vol = sum(x[4] for x in data)
        buy_order = sum(x[6] for x in data)
        
        # if can't figure it out in 1st minute, change buy_vol to 10 and buy_order to 100
        if table not in skip_list and buy_vol > 1 and buy_order > 50:
            buy_volume[table] = (buy_vol, buy_order)

    sleep(1)

print(datetime.now())
pprint(sorted(buy_volume.items(), key=itemgetter(1), reverse=True))

2017-12-21 13:11:43.222284
[('BTC-XRP', (Decimal('321.12455027'), 3526)),
 ('BTC-WAVES', (Decimal('73.51618828'), 1078)),
 ('BTC-STRAT', (Decimal('47.50629196'), 815)),
 ('BTC-ADA', (Decimal('40.95357169'), 748)),
 ('BTC-GBG', (Decimal('40.20399092'), 1274)),
 ('BTC-XZC', (Decimal('37.81629239'), 1159)),
 ('BTC-XEM', (Decimal('36.80775029'), 442)),
 ('BTC-NXT', (Decimal('34.60780841'), 685)),
 ('BTC-DOGE', (Decimal('31.39759475'), 441)),
 ('BTC-XMR', (Decimal('26.50849901'), 721)),
 ('BTC-QTUM', (Decimal('23.59889891'), 425)),
 ('BTC-VTC', (Decimal('22.10826533'), 457)),
 ('BTC-DASH', (Decimal('21.43153433'), 551)),
 ('BTC-SC', (Decimal('20.32273341'), 494)),
 ('BTC-BTG', (Decimal('17.33261075'), 337)),
 ('BTC-MCO', (Decimal('17.12354857'), 474)),
 ('BTC-PAY', (Decimal('15.48440725'), 251)),
 ('BTC-NEO', (Decimal('14.48989857'), 347)),
 ('BTC-ZEC', (Decimal('14.45056252'), 512)),
 ('BTC-DGB', (Decimal('14.15988361'), 353)),
 ('BTC-LSK', (Decimal('13.23507563'), 245)),
 ('BTC-KMD', (Dec

In [268]:
market = ''

In [175]:
wallet = 0

price = b.get_ticker(market).get('result').get('Ask')
buy_price = float(Decimal(price * 1.1).quantize(Decimal(10) ** -8))
buy_volume = int(wallet / buy_price)

In [158]:
b.buy_limit(market, buy_volume, buy_price)

{'message': 'INSUFFICIENT_FUNDS', 'result': None, 'success': False}